In [2]:
# Install required packages 
!pip install nltk

In [3]:
!pip install newspaper3k

In [4]:
!pip install lxml[html_clean]

In [6]:
pip install --upgrade threadpoolctl

  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)
  Attempting uninstall: threadpoolctl
    Found existing installation: threadpoolctl 2.2.0
    Uninstalling threadpoolctl-2.2.0:
      Successfully uninstalled threadpoolctl-2.2.0
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install --upgrade scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [1]:
# Import necessary libraries
from newspaper import Article
import random
import string
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [5]:
# Download the punkt package
nltk.download('punkt', quiet=True)
print("Loading article content... Please wait.")

try:
    # Get the article
    article = Article('https://www.mayoclinic.org/diseases-conditions/chronic-kidney-disease/symptoms-causes/syc-20354521')
    article.download()
    article.parse()
    article.nlp()
    corpus = article.text
    print("Article loaded successfully!")
    print(f"Article length: {len(corpus)} characters")
    print("-" * 50)
    
except Exception as e:
    print(f"Error loading article: {e}")

Loading article content... Please wait.
Article loaded successfully!
Article length: 6382 characters
--------------------------------------------------


In [6]:
# Tokenization
text = corpus
sentence_list = nltk.sent_tokenize(text)  # A list of sentences

print(f"Processed {len(sentence_list)} sentences from the article.")


Processed 71 sentences from the article.


In [7]:
# A function to return a random greeting response to a user's greeting 
def greeting_response(text):
    text = text.lower()
    # Bot's greeting response
    bot_greetings = ['howdy', 'hi', 'hey', 'hello', 'hola', 'greetings!', 'nice to meet you!']
    # User's greeting patterns
    user_greetings = ['hi', 'hey', 'hello', 'hola', 'greetings', 'wassup', 'good morning', 'good afternoon', 'good evening']
    
    for word in text.split():
        if word in user_greetings:
            return random.choice(bot_greetings)
    return None

In [8]:
# Fixed sorting function
def index_sort(list_var):
    length = len(list_var)
    list_index = list(range(0, length))
    
    x = list_var
    for i in range(length):
        for j in range(length):
            if x[list_index[i]] > x[list_index[j]]:
                # Fixed: swap both indices properly
                temp = list_index[i]
                list_index[i] = list_index[j]
                list_index[j] = temp  # FIXED: was missing [j]
    return list_index

In [9]:
# Enhanced bot response function
def bot_response(user_input):
    user_input = user_input.lower()
    sentence_list.append(user_input)
    bot_response = ''
    
    try:
        cm = CountVectorizer().fit_transform(sentence_list)
        similarity_scores = cosine_similarity(cm[-1], cm)
        similarity_scores_list = similarity_scores.flatten()
        index = index_sort(similarity_scores_list)
        index = index[1:]  # Remove the user input itself
        response_flag = 0
        
        j = 0
        for i in range(len(index)):
            if similarity_scores_list[index[i]] > 0.0:
                bot_response = bot_response + ' ' + sentence_list[index[i]]
                response_flag = 1
                j = j + 1
            if j >= 2:  # FIXED: was j<2, now j>=2 to break after getting 2 responses
                break
                
        if response_flag == 0:
            bot_response = bot_response + ' ' + "I apologize, I don't understand. Could you please rephrase your question about chronic kidney disease?"
            
    except Exception as e:
        bot_response = "I'm having trouble processing your question. Please try asking about chronic kidney disease symptoms, causes, or treatment."
    
    sentence_list.remove(user_input)
    return bot_response.strip()

In [10]:
# Enhanced chat interface
def start_chat():
    print('=' * 60)
    print('🤖 Doc Bot: Hello! I am Doctor Bot (Doc Bot for short).')
    print('   I can answer your questions about Chronic Kidney Disease.')
    print('   Ask me about symptoms, causes, treatments, or prevention.')
    print('   Type "bye", "exit", or "quit" to end our conversation.')
    print('=' * 60)
    
    exit_list = ['exit', 'see you later', 'bye', 'quit', 'break', 'goodbye']
    
    while True:
        try:
            print("\n💬 You: ", end="")
            user_input = input().strip()
            
            if not user_input:
                print("🤖 Doc Bot: Please say something! I'm here to help with kidney disease questions.")
                continue
                
            if user_input.lower() in exit_list:
                print('🤖 Doc Bot: Thank you for chatting! Take care of your health. Goodbye! 👋')
                break
            else:
                greeting_resp = greeting_response(user_input)
                if greeting_resp is not None:
                    print(f'🤖 Doc Bot: {greeting_resp} How can I help you with chronic kidney disease today?')
                else:
                    response = bot_response(user_input)
                    print(f'🤖 Doc Bot: {response}')
                    
        except KeyboardInterrupt:
            print('\n🤖 Doc Bot: Chat interrupted. Goodbye!')
            break
        except Exception as e:
            print(f'🤖 Doc Bot: Sorry, I encountered an error. Please try again.')

In [11]:
# Start the chatbot
if __name__ == "__main__":
    start_chat()

🤖 Doc Bot: Hello! I am Doctor Bot (Doc Bot for short).
   I can answer your questions about Chronic Kidney Disease.
   Ask me about symptoms, causes, treatments, or prevention.
   Type "bye", "exit", or "quit" to end our conversation.

💬 You: 

 hii


🤖 Doc Bot: I apologize, I don't understand. Could you please rephrase your question about chronic kidney disease?

💬 You: 

 What is chronic kidney disease?


🤖 Doc Bot: Chronic kidney disease is a disease characterized by progressive damage and loss of function in the kidneys. What is it?

💬 You: 

...

🤖 Doc Bot: If you have kidney disease, there are lifestyle choices. Learning you have chronic kidney disease and learning how to live with it can be a challenge.

💬 You: 

 Can kidney disease be prevented?


🤖 Doc Bot: Smoking can also be a contributing factor to chronic kidney disease. Learning you have chronic kidney disease and learning how to live with it can be a challenge.

💬 You: 

 bye


🤖 Doc Bot: Thank you for chatting! Take care of your health. Goodbye! 👋
